In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 1.X

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.X`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
#Installing BERT module
!pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 71kB 3.2MB/s 


## Import libraries

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

tensorflow version :  1.15.2
tensorflow_hub version :  0.10.0



## Set parameters

In [5]:
# Set the output directory for saving model file
OUTPUT_DIR = 'model'

label_list = [0, 1, 2]
# We will also set a sequence length which will be the length of the input features.
MAX_SEQ_LENGTH = 256 #128

## Prepare the model

The following code block loads the pre-trained BERT model and initializers a tokenizer object for tokenizing the texts.  
check the model list here: https://tfhub.dev/s?network-architecture=transformer&publisher=google

In [6]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## Create multi-class classifier model

In [7]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [8]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [9]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = 80
num_warmup_steps = 8

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [10]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f16e6ac59b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f16e6ac59b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Load the trained checkpoint

In [13]:
!cp drive/My\ Drive/Colab\ data/BERT/BERTmodel.tar.gz .
!tar -xf BERTmodel.tar.gz

## Predict function

In [23]:
# low medium high
# 0 1 2
# A method to get predictions
def getPrediction(in_sentences):
  #A list to map the actual labels to the predictions
  labels = ['low', 'high', 'None']

  #Transforming the test data into BERT accepted form
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  
  #Creating input features for Test data
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

## Predict the csv file (multiple inputs)

In [26]:
#multiple inputs
#用記事本打開csv文件，另存為设置編碼為utf-8
test_data_source = "data/test_data_temp.csv"
test_data_df = pd.read_csv(test_data_source, encoding='utf8', delimiter=',')
print(test_data_df.head())

print("\n")
test_data = list(test_data_df['Description'])
print(test_data[0])

   ID                                        Description  Class
0   0  - - - - - - - - - - - - - -附錄：【發文與推文罰則】- - - -...    NaN
1   1                     --　　　　　　　　　　　　　　　　　　　　　    　--    NaN
2   2  版友們有哪些討厭的副作用呢？今天去運動途中哭了因為副作用讓我覺得沒辦法掌控自己的身體很痛苦 ...    NaN
3   3  「我會在，你放心。我們就慢慢把環境適應。」治療的最後，L說著，想要安撫我下週工作室搬遷的焦慮...    NaN
4   4  其實我也不知道在焦慮甚麼也許是不想有人來管吧?本來是社區自聘的管理員6/1起外包給保全公司我...    NaN


- - - - - - - - - - - - - -附錄：【發文與推文罰則】- - - - - - - - - - - - - -第一條　凡未分類、自創分類、未滿20字（不含標點符號）之文章，　　　　含有自殺、自殘、OD相關內容，未在標題加註「有雷」或「雷」者，　　　　初犯者板主將直接刪除文章，不另外公告或通知。　　　　其後發文或推文仍未改善者劣退水桶一個月。三犯永久水桶。第二條　發文內容或推文大部分內容，違反中華民國法律者，　　　　初犯者板主將直接刪除文章並並水桶三日，不另外公告或通知。　　　　其後發文或推文仍未改善者劣退水桶一個月。三犯永久水桶。第三條　尋死文、自殺文、自殘文、OD文、藥物徵求買賣文等違規文章，　　　　除自殺文暫保留二日以便板友關心追蹤外，其餘文直接刪除並水桶三日。　　　　解除水桶之後其發文仍未遵循板規者，其發文劣退並水桶一個月。　　　　水桶後無論自行發文或在其他板友文章內推文再犯者皆永久水桶。第四條　本板對人身攻擊定義，不論是主動攻擊他人或因對他人言論不滿而回擊者，　　　　只要構成人身攻擊之事實，即視為違規，依程度輕重照板規懲處。　　　　此外，意見不合與人身攻擊間有很大的落差，　　　　請發文推文者勿因他人持反對意見，就任意指責對方人身攻擊，　　　　如經板主認定檢舉人有濫訴之嫌，得以不回應其檢舉。　　　　　　　認定標準為：該字詞足以引起本板板友普遍性之不悅，並達到動怒的程度。　　　　例：

In [18]:
test_results = getPrediction(test_data)
print("#"*50)
print(test_results[0])

INFO:tensorflow:Writing example 0 of 11


INFO:tensorflow:Writing example 0 of 11


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


##################################################
('- - - - - - - - - - - - - -附錄：【發文與推文罰則】- - - - - - - - - - - - - -第一條\u3000凡未分類、自創分類、未滿20字（不含標點符號）之文章，\u3000\u3000\u3000\u3000含有自殺、自殘、OD相關內容，未在標題加註「有雷」或「雷」者，\u3000\u3000\u3000\u3000初犯者板主將直接刪除文章，不另外公告或通知。\u3000\u3000\u3000\u3000其後發文或推文仍未改善者劣退水桶一個月。三犯永久水桶。第二條\u3000發文內容或推文大部分內容，違反中華民國法律者，\u3000\u3000\u3000\u3000初犯者板主將直接刪除文章並並水桶三日，不另外公告或通知。\u3000\u3000\u3000\u3000其後發文或推文仍未改善者劣退水桶一個月。三犯永久水桶。第三條\u3000尋死文、自殺文、自殘文、OD文、藥物徵求買賣文等違規文章，\u3000\u3000\u3000\u3000除自殺文暫保留二日以便板友關心追蹤外，其餘文直接刪除並水桶三日。\u3000\u3000\u3000\u3000解除水桶之後其發文仍未遵循板規者，其發文劣退並水桶一個月。\u3000\u3000\u3000\u3000水桶後無論自行發文或在其他板友文章內推文再犯者皆永久水桶。第四條\u3000本板對人身攻擊定義，不論是主動攻擊他人或因對他人言論不滿而回擊者，\u3000\u3000\u3000\u3000只要構成人身攻擊之事實，即視為違規，依程度輕重照板規懲處。\u3000\u3000\u3000\u3000此外，意見不合與人身攻擊間有很大的落差，\u3000\u3000\u3000\u3000請發文推文者勿因他人持反對意見，就任意指責對方人身攻擊，\u3000\u3000\u3000\u3000如經板主認定檢舉人有濫訴之嫌，得以不回應其檢舉。\u3000\u3000\u3000\u3000\u3000\u3000\u3000認定標準為：該字詞足以引起本板板友普遍性之不悅，並達到動怒的程度。\u3000\u3000\u3000\u3000例：「你根本不是憂鬱症吧！」、「會生病是

In [19]:
results = [tr[2] for tr in test_results]
print(results)
# save the result back to the csv file
test_data_df['Predicted'] = results
test_data_df.to_csv('data/test_data_result.csv', encoding='utf8')

[0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]


## Predict single input here

In [24]:
#Classifying random sentences
test_results = getPrediction(['其實我本來就不太想活了每次和媽媽說完話我就開始辜狗自殺方法',
                       '帶了很多藥來就是因為怕這樣的時刻又出現這才是真正的逢魔時刻吧想自我毀滅盡量躺著保持安靜再撐幾天就回家了沒事的',
                       '還有半小時覺得倦怠低落負面而自我放逐下班要吃冰冰彷彿是每天的救贖潰瘍還沒好',
                       '群組裡面都是很負面的話受不了就不要加入下輩子再見也可以說不定會好 '
                       ])

INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from model/model.ckpt-70


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [25]:
test_results

[('其實我本來就不太想活了每次和媽媽說完話我就開始辜狗自殺方法',
  array([-3.9395003 , -0.02154459, -6.289239  ], dtype=float32),
  1,
  'high'),
 ('帶了很多藥來就是因為怕這樣的時刻又出現這才是真正的逢魔時刻吧想自我毀滅盡量躺著保持安靜再撐幾天就回家了沒事的',
  array([-4.360148  , -0.01549961, -5.9508667 ], dtype=float32),
  1,
  'high'),
 ('還有半小時覺得倦怠低落負面而自我放逐下班要吃冰冰彷彿是每天的救贖潰瘍還沒好',
  array([-3.9878404 , -0.02051002, -6.3416457 ], dtype=float32),
  1,
  'high'),
 ('群組裡面都是很負面的話受不了就不要加入下輩子再見也可以說不定會好 ',
  array([-1.7387948 , -0.19585973, -6.1467667 ], dtype=float32),
  1,
  'high')]

Reference: https://analyticsindiamag.com/step-by-step-guide-to-implement-multi-class-classification-with-bert-tensorflow/